# Markovify

This code is for training and testing the markov chain model for word recommendation

In [224]:
import markovify
import re
import spacy
import random

!python -m spacy download en_core_web_sm

2023-04-24 23:11:45.432094: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Training the markov model based on our training dataset

In [225]:
with open("../data/output_train.txt") as f:
    text = f.read()
    
with open("../data/output_train.txt") as f:
    test_text = f.read().split("\n")

training_model = markovify.NewlineText(text, state_size=3)


Testing the initial markov model for correct prediction

In [239]:
correct = 0.0
total_runs = 5000
completed_runs = 0

for i in range(total_runs):
    rand = random.randint(0, len(test_text))
    test_sentence = test_text[rand].split()
    start_words = ' '.join(test_sentence[:2])
    
    try:
        output = training_model.make_sentence_with_start(start_words, max_chars=180)
        if output.split()[2] == test_sentence[2]:
            correct += 1
        completed_runs += 1
    except:
        continue

print(f'Correct: {correct / total_runs}')
print(f'Completed runs: {(completed_runs - correct) / total_runs}')
print(f'Invalid runs: {(total_runs - completed_runs) / total_runs}')

Correct: 0.095
Completed runs: 0.1408
Invalid runs: 0.7642


Now we'll test spaCy's POS tagging with markovify to see if it improves text predictions.

In [227]:
nlp = spacy.load("en_core_web_sm")

class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

    def sentence_split(self, text):
        return re.split(r"\s*\n\s*", text)

In [228]:
training_model_2 = POSifiedText(text, state_size=3)

In [240]:
correct = 0.0
completed_runs = 0

for i in range(total_runs):
    rand = random.randint(0, len(test_text))
    test_sentence = test_text[rand].split()
    start_words = ' '.join(test_sentence[:2])
    
    try:
        output = training_model_2.make_sentence_with_start(start_words, max_chars=180)
        if output.split()[2] == test_sentence[2]:
            correct += 1
        completed_runs += 1
    except:
        continue

print(f'Correct: {correct / total_runs}')
print(f'Completed runs: {(completed_runs - correct) / total_runs}')
print(f'Invalid runs: {(total_runs - completed_runs) / total_runs}')

Correct: 0.0632
Completed runs: 0.18
Invalid runs: 0.7568
